Bioimpedance Exploration Notebook

In [191]:

##### Bioimpedance Exloration for Exo Fatigue protocol #####
# import functions 
import pandas as pd 
from matplotlib import use
use("TkAgg")
import matplotlib.pyplot as plt
import numpy as np
import json
from os.path import exists
from os import makedirs
# import sklearn
import csv
import pickle
import sklearn.decomposition as dc
from Utilities.Dataset import parse_file_for_data_sets
from Utilities.AdvancedLoggingParser import *
from datetime import datetime

name - cateogory number
q1 (vest) = 7
s6_exo = 8
q2 (base) = 9
s7base = 10
s8exo = 11
q3base = 12
s8base = 13
q3vest = 14
s7exo = 15
q4base = 16
q4vest = 17
q6vest = 18
q7vest = 19
q8base = 20
q7base = 21
q8vest = 22
s9base = 23
s9exo = 24
s10base = 25
s11exo = 26
s11base = 27

In [58]:
# Testing JSON parser
master_sync_file = open('Data/kinematicsData/exo_study_amro/timestamper_sync_master_24hr.json')

data = json.load(master_sync_file)
data

{'category': [{'cate_id': 1, 'cate_name': 'All', 'cate_image': 'all'},
  {'cate_id': 2, 'cate_name': 'Default', 'cate_image': 'category'},
  {'cate_id': 3, 'cate_name': 'Baby', 'cate_image': 'baby'},
  {'cate_id': 4, 'cate_image': 'pills', 'cate_name': 'Pills'},
  {'cate_id': 5, 'cate_name': 'Travels', 'cate_image': 'travels'},
  {'cate_id': 6, 'cate_name': 'Gym', 'cate_image': 'gym'},
  {'cate_id': 7, 'cate_image': '', 'cate_name': 'q1'},
  {'cate_id': 8, 'cate_name': 's6_exo', 'cate_image': ''},
  {'cate_id': 9, 'cate_name': 'q2', 'cate_image': ''},
  {'cate_id': 10, 'cate_name': 's7base', 'cate_image': ''},
  {'cate_id': 11, 'cate_image': '', 'cate_name': 's8exo'},
  {'cate_id': 12, 'cate_image': '', 'cate_name': 'q3base'},
  {'cate_id': 13, 'cate_name': 's8base', 'cate_image': ''},
  {'cate_id': 14, 'cate_image': '', 'cate_name': 'q3vest'},
  {'cate_id': 15, 'cate_name': 's7exo', 'cate_image': ''},
  {'cate_id': 16, 'cate_name': 'q4base', 'cate_image': ''},
  {'cate_id': 17, 'cate_

In [187]:
data.keys()
# data
sorted = pd.DataFrame()

for ii in data['note']:
    
    if ii['cat_id'] == 7: #or ii['cat_id'] == 2:
        if sorted.empty:
            sorted = pd.DataFrame({'id':ii['cat_id'],'time':[ii['time'][:-3]+'.' + ii['time'][-2:] + '0000'], 'dateNtime':[ii['dateNtime'][-8:]], 'ak_sync':[0], 'kn_sync':[0], 'note_desc':ii['note_desc']})
        else:
            temp_df = pd.DataFrame({'id':ii['cat_id'],'time':[ii['time'][:-3]+'.' + ii['time'][-2:] + '0000'], 'dateNtime':[ii['dateNtime'][-8:]], 'ak_sync':[0], 'kn_sync':[0], 'note_desc':ii['note_desc']})
            sorted = pd.concat([sorted, temp_df])

    # sorted = {key: value for key, value in }
    # print(sorted_three)

sorted.reset_index(drop = True, inplace=True)
sorted[:20]


,id,time,dateNtime,ak_sync,kn_sync,note_desc
0,2,17:19:16.940000,17191694,0,0,
1,2,18:41:04.400000,18410440,0,0,
2,2,18:05:04.080000,18050408,0,0,
3,7,18:31:18.730000,18311873,0,0,
4,7,18:32:17.270000,18321727,0,0,
5,7,18:37:23.980000,18372398,0,0,
6,7,18:42:22.370000,18422238,0,0,
7,7,18:47:18.510000,18471852,0,0,
8,7,18:52:23.010000,18522302,0,0,
9,7,18:57:29.640000,18572965,0,0,


In [188]:
# For calculating sync

def convert_timedelta(duration):
    days, seconds = duration.days, duration.seconds
    hours = days * 24 + seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = (seconds % 60)
    return hours, minutes, seconds

kn_sync_start_idx = 2
ak_sync_start_idx = 2


for jj in range(len(sorted)):

    # knee first
    if jj > kn_sync_start_idx:
        t1 = datetime.strptime(sorted['time'].iloc[kn_sync_start_idx], "%H:%M:%S.%f")
        t2 = datetime.strptime(sorted['time'].iloc[jj], "%H:%M:%S.%f")
        delta = t2 - t1
        sorted.loc[sorted.index[jj], 'kn_sync'] = str(delta)
        # sorted['kn_sync'].iloc[jj] = str(delta)
    else:
        sorted.loc[sorted.index[jj], 'kn_sync'] = "0:00:00.000000"
        # sorted.iloc[jj, sorted.columns.get_loc('kn_sync')] = "0:00:00.000000"


    if jj > ak_sync_start_idx:
        t1 = datetime.strptime(sorted['time'].iloc[ak_sync_start_idx], "%H:%M:%S.%f")
        t2 = datetime.strptime(sorted['time'].iloc[jj], "%H:%M:%S.%f")
        delta = t2 - t1
        sorted.loc[sorted.index[jj], 'ak_sync'] = str(delta)
        # sorted['ak_sync'].iloc[jj] = str(delta)
    else:
        sorted.loc[sorted.index[jj], 'ak_sync'] = "0:00:00.000000"
        # sorted['ak_sync'].iloc[jj] = "0:00:00.000000"


# sorted = sorted.iloc[2:,:]




In [189]:
sorted

,id,time,dateNtime,ak_sync,kn_sync,note_desc
2,2,18:05:04.080000,18050408,0:00:00.000000,0:00:00.000000,
3,7,18:31:18.730000,18311873,0:26:14.650000,0:26:14.650000,
4,7,18:32:17.270000,18321727,0:27:13.190000,0:27:13.190000,
5,7,18:37:23.980000,18372398,0:32:19.900000,0:32:19.900000,
6,7,18:42:22.370000,18422238,0:37:18.290000,0:37:18.290000,
7,7,18:47:18.510000,18471852,0:42:14.430000,0:42:14.430000,
8,7,18:52:23.010000,18522302,0:47:18.930000,0:47:18.930000,
9,7,18:57:29.640000,18572965,0:52:25.560000,0:52:25.560000,
10,7,19:02:34.280000,19023429,0:57:30.200000,0:57:30.200000,


In [192]:
def save_sync_csv(df, filename, filedir = 'Data/kinematicsData/exo_study_amro/sync_csv'):
    dirbio = filedir + '/'+filename + '_syncs.csv'
    df.to_csv(dirbio)


In [190]:
save_sync_csv(sorted, filename='q1_vest')

In [193]:
def save_csv(bioz_df, imu_df, filename, filedir = 'Data/kinematicsData/exo_study_amro/csv_data_new'):
    dirbio = filedir + '/'+filename + '_bioz.csv'
    dirimu = filedir + '/'+filename + '_imu.csv'
    bioz_df.to_csv(dirbio)
    imu_df.to_csv(dirimu)

In [11]:
# do this if wanting to plot inline
# %matplotlib inline

In [194]:
# save datasets to pickle file for speed 
def save_data(data, fileLoc):
    slash = fileLoc.rfind('/')
    name = fileLoc[slash+1:]
    savelocation = 'Data/kinematicsData/exo_study_data/'+name+'.pkl'
    with open(savelocation, 'wb') as f:
        pickle.dump(data, f) 
    return

In [195]:
def preprocess_bioZ(fileLoc, filenum = -1, name = 'temp', exo_status = 0, joint = 'ankle', new = 0, subject = None):
    if new == 1: # if data is coming from new file parsing on new device
        if subject == None:
            subject = 2 #state subject num

        dataset = AdvancedLoggingDataset(fileLoc)
        # dataset.BioZ1.ElectrodeImpedance # vector length of bioz, 1d
        # dataset.BioZ1.Data # same format as before: freq, datapoint, tt r or x
        # arrange into dataframe
        temp_dict = {'tt5k':dataset.BioZ1.Data[0,:,0]/60000, 'tt100k':dataset.BioZ1.Data[-1,:,0]/60000, 'R5k': dataset.BioZ1.Data[0,:,1], 'R100k': dataset.BioZ1.Data[-1,:,1],
                        'X5k': dataset.BioZ1.Data[0,:,2], 'X100k': dataset.BioZ1.Data[-1,:,2], 'electZ5k':dataset.BioZ1.ElectrodeImpedance, 'electZ100k':dataset.BioZ1.ElectrodeImpedance}
        bioz_df = pd.DataFrame(temp_dict)
        imu_df = pd.DataFrame(dataset.IMU.Data, columns=['tt','acc_x1', 'acc_y1', 'acc_z1', 'gyro_x1', 'gyro_y1', 'gyro_z1',
                                                            'acc_x2', 'acc_y2', 'acc_z2', 'gyro_x2', 'gyro_y2', 'gyro_z2'])
        imu_df['tt'] /= 60000
        # add identifier data to the dataframes
        imu_df.insert(0, "subject", subject * np.ones_like(dataset.IMU.Data[:,0]))
        bioz_df.insert(0, "subject", subject * np.ones_like(dataset.BioZ1.Data[0,:,0]))   
        imu_df.insert(1, "name", np.repeat(name, len(dataset.IMU.Data[:,0])))
        bioz_df.insert(1, "name", np.repeat(name, len(dataset.BioZ1.Data[0,:,0])))
        imu_df.insert(2, "exo", exo_status * np.ones_like(dataset.IMU.Data[:,0]))
        bioz_df.insert(2, "exo", exo_status * np.ones_like(dataset.BioZ1.Data[0,:,0]))
        imu_df.insert(3, "joint", np.repeat(joint, len(dataset.IMU.Data[:,0])))
        bioz_df.insert(3, "joint", np.repeat(joint, len(dataset.BioZ1.Data[0,:,0])))
        imu_df.insert(4, "file", 0 * np.ones_like(dataset.IMU.Data[:,0]))
        bioz_df.insert(4, "file", 0 * np.ones_like(dataset.BioZ1.Data[0,:,0]))
        return bioz_df, imu_df
    
    
    else: # go through old parser
        GFCoefficientsTIA = np.load("Data/CalibrationData/B001/TIACoeff.npy")
        if exists(fileLoc+'.pkl'):
            Datasets =  pickle.load(open(fileLoc+'.pkl', 'rb')) 
        else:
            Datasets = parse_file_for_data_sets(fileLoc +'.txt')
            # save_data(Datasets, fileLoc)
        if len(Datasets) != 1:
            print(len(Datasets))
        if subject == None:
            # subject = 3 # state subject num
            subject = int(fileLoc[44])
        
        bioz_df_all = pd.DataFrame()
        imu_df_all = pd.DataFrame()
        
        # Do this option if trying to get single file - i.e. multiple subjects on one file
        if filenum != -1: 
            dataset = Datasets[filenum]
            NumFrequencies = int(dataset.JsonDict["BioZ"]["NumFrequencies"])
            Frequencies = np.linspace(5000, 100000, NumFrequencies)
            Zprotection = (1/1j*2*np.pi*Frequencies*0.01e-6)+(1/1j*2*np.pi*Frequencies*0.47e-6) + 2000
            #generate raw current and voltage vectors
            Vv = np.zeros([dataset.RawDataset.BioZ.shape[1],NumFrequencies], dtype=complex)
            Vi = np.zeros([dataset.RawDataset.BioZ.shape[1],NumFrequencies],dtype=complex)
            for i in range(dataset.RawDataset.BioZ.shape[1]):
                Vv[i] = dataset.RawDataset.CurrentVoltage[:,i,2]# - VvBias
                Vi[i] = dataset.RawDataset.CurrentVoltage[:,i,1] #- ViBias
            Vtinv = 1/Vi
            Vz = Vv * Vtinv
            AtiaGF = (np.zeros([Vz.shape[0], 2, NumFrequencies]))
            ZtotEstMag = np.zeros(Vz.shape)
            
            #calibrate data
            for i in range(NumFrequencies):
                AtiaGF[:,0,i] = np.abs(Vtinv[:,i])
                AtiaGF[:,1,i] = 1
                ZtotEstMag[:, i] = np.dot(AtiaGF[:, :, i], GFCoefficientsTIA[:, i]) - np.abs(Zprotection[i])
            
            # arrange into dataframe
            temp_dict = {'tt5k':dataset.BioZ.Data[0,:,0]/60000,'tt100k':dataset.BioZ.Data[-1,:,0]/60000, 'R5k': dataset.BioZ.Data[0,:,1], 'R100k': dataset.BioZ.Data[-1,:,1],
                            'X5k': dataset.BioZ.Data[0,:,2], 'X100k': dataset.BioZ.Data[-1,:,2], 'electZ5k':ZtotEstMag[:,0], 'electZ100k':ZtotEstMag[:,-1]}
            bioz_df = pd.DataFrame(temp_dict)
            imu_df = pd.DataFrame(dataset.IMU.Data, columns=['tt','acc_x1', 'acc_y1', 'acc_z1', 'gyro_x1', 'gyro_y1', 'gyro_z1',
                                                                'acc_x2', 'acc_y2', 'acc_z2', 'gyro_x2', 'gyro_y2', 'gyro_z2'])
            imu_df['tt'] /= 60000
            # add identifier data to the dataframes
            imu_df.insert(0, "subject", subject * np.ones_like(dataset.IMU.Data[:,0]))
            bioz_df.insert(0, "subject", subject * np.ones_like(dataset.BioZ.Data[0,:,0]))   
            imu_df.insert(1, "name", np.repeat(name, len(dataset.IMU.Data[:,0])))
            bioz_df.insert(1, "name", np.repeat(name, len(dataset.BioZ.Data[0,:,0])))
            imu_df.insert(2, "exo", exo_status * np.ones_like(dataset.IMU.Data[:,0]))
            bioz_df.insert(2, "exo", exo_status * np.ones_like(dataset.BioZ.Data[0,:,0]))
            imu_df.insert(3, "joint", np.repeat(joint, len(dataset.IMU.Data[:,0])))
            bioz_df.insert(3, "joint", np.repeat(joint, len(dataset.BioZ.Data[0,:,0])))
            imu_df.insert(4, "file", filenum * np.ones_like(dataset.IMU.Data[:,0]))
            bioz_df.insert(4, "file", filenum * np.ones_like(dataset.BioZ.Data[0,:,0]))
            return bioz_df, imu_df
        else:    
            # dataset = Datasets[filenum]
            timehold5k = 0
            timehold100k = 0
            timeholdimu = 0
 
            for count, dataset in enumerate(Datasets):
                NumFrequencies = int(dataset.JsonDict["BioZ"]["NumFrequencies"])
                Frequencies = np.linspace(5000, 100000, NumFrequencies)
                Zprotection = (1/1j*2*np.pi*Frequencies*0.01e-6)+(1/1j*2*np.pi*Frequencies*0.47e-6) + 2000
                #generate raw current and voltage vectors
                Vv = np.zeros([dataset.RawDataset.BioZ.shape[1],NumFrequencies], dtype=complex)
                Vi = np.zeros([dataset.RawDataset.BioZ.shape[1],NumFrequencies],dtype=complex)
                for i in range(dataset.RawDataset.BioZ.shape[1]):
                    Vv[i] = dataset.RawDataset.CurrentVoltage[:,i,2]# - VvBias
                    Vi[i] = dataset.RawDataset.CurrentVoltage[:,i,1] #- ViBias
                Vtinv = 1/Vi
                Vz = Vv * Vtinv
                AtiaGF = (np.zeros([Vz.shape[0], 2, NumFrequencies]))
                ZtotEstMag = np.zeros(Vz.shape)
                
                #calibrate data
                for i in range(NumFrequencies):
                    AtiaGF[:,0,i] = np.abs(Vtinv[:,i])
                    AtiaGF[:,1,i] = 1
                    ZtotEstMag[:, i] = np.dot(AtiaGF[:, :, i], GFCoefficientsTIA[:, i]) - np.abs(Zprotection[i])
                
                # arrange into dataframe
                temp_dict = {'tt5k':dataset.BioZ.Data[0,:,0]/60000,'tt100k':dataset.BioZ.Data[-1,:,0]/60000, 'R5k': dataset.BioZ.Data[0,:,1], 'R100k': dataset.BioZ.Data[-1,:,1],
                                'X5k': dataset.BioZ.Data[0,:,2], 'X100k': dataset.BioZ.Data[-1,:,2], 'electZ5k':ZtotEstMag[:,0], 'electZ100k':ZtotEstMag[:,-1]}
                bioz_df = pd.DataFrame(temp_dict)
                imu_df = pd.DataFrame(dataset.IMU.Data, columns=['tt','acc_x1', 'acc_y1', 'acc_z1', 'gyro_x1', 'gyro_y1', 'gyro_z1',
                                                                    'acc_x2', 'acc_y2', 'acc_z2', 'gyro_x2', 'gyro_y2', 'gyro_z2'])
                imu_df['tt'] /= 60000
                bioz_df['tt5k'] += timehold5k
                bioz_df['tt100k'] += timehold100k
                imu_df['tt'] += timeholdimu

                # add identifier data to the dataframes
                imu_df.insert(0, "subject", subject * np.ones_like(dataset.IMU.Data[:,0]))
                bioz_df.insert(0, "subject", subject * np.ones_like(dataset.BioZ.Data[0,:,0]))   
                imu_df.insert(1, "name", np.repeat(name, len(dataset.IMU.Data[:,0])))
                bioz_df.insert(1, "name", np.repeat(name, len(dataset.BioZ.Data[0,:,0])))
                imu_df.insert(2, "exo", exo_status * np.ones_like(dataset.IMU.Data[:,0]))
                bioz_df.insert(2, "exo", exo_status * np.ones_like(dataset.BioZ.Data[0,:,0]))
                imu_df.insert(3, "joint", np.repeat(joint, len(dataset.IMU.Data[:,0])))
                bioz_df.insert(3, "joint", np.repeat(joint, len(dataset.BioZ.Data[0,:,0])))
                imu_df.insert(4, "file", count * np.ones_like(dataset.IMU.Data[:,0]))
                bioz_df.insert(4, "file", count * np.ones_like(dataset.BioZ.Data[0,:,0]))
                bioz_df_all = pd.concat([bioz_df_all, bioz_df])
                imu_df_all = pd.concat([imu_df_all, imu_df])
                
                #index time offset
                timehold5k += max(bioz_df.tt5k)
                timehold100k += max(bioz_df.tt100k)
                timeholdimu += max(imu_df.tt)

                del bioz_df, imu_df
            return bioz_df_all, imu_df_all

In [199]:
# bioz_df, imu_df = preprocess_bioZ('Data/kinematicsData/exo_study_amro/biomech_s5_base', filenum = -1, name = 'jacque', exo_status = 0, joint = 'ankle')
bioz_df, imu_df = preprocess_bioZ('Data/kinematicsData/exo_study_amro/s1_ak_exo', filenum = 2, name = 'christof', exo_status = 1, joint = 'ankle', new = 0, subject = 1)
# bioz_df, imu_df = preprocess_bioZ('Data/kinematicsData/exo_study_amro/biomech_s6_exo', filenum = -1, name = 'lindsey', exo_status = 1, joint = 'ankle', new = 0)



save_csv(bioz_df, imu_df, filename='s1_ak_exo')
imu_df


3


,subject,name,exo,joint,file,tt,acc_x1,acc_y1,acc_z1,gyro_x1,gyro_y1,gyro_z1,acc_x2,acc_y2,acc_z2,gyro_x2,gyro_y2,gyro_z2
0,1.0,christof,1.0,ankle,2.0,0.056274,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1.0,christof,1.0,ankle,2.0,0.056608,-0.961914,-0.094910,-0.076233,0.136108,-0.073853,-0.034058,0.356567,-0.583008,0.742493,-0.093323,-0.202515,-0.057068
2,1.0,christof,1.0,ankle,2.0,0.056942,-0.969727,-0.095215,-0.078430,0.055542,-0.027405,0.026001,0.362732,-0.599854,0.735291,-0.067749,-0.157593,0.019409
3,1.0,christof,1.0,ankle,2.0,0.057275,-0.969421,-0.103760,-0.091187,0.022888,-0.007141,-0.008850,0.371155,-0.613220,0.719055,0.008545,0.002625,-0.004761
4,1.0,christof,1.0,ankle,2.0,0.057609,-0.971619,-0.118530,-0.088745,0.021790,-0.006104,-0.009460,0.382629,-0.622559,0.707458,0.008606,0.002686,-0.004822
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121845,1.0,christof,1.0,ankle,2.0,43.387736,-0.660461,-0.722534,-0.113098,0.005066,-0.006165,-0.005554,0.469666,-0.586060,0.706665,0.014099,-0.000122,-0.001709
121846,1.0,christof,1.0,ankle,2.0,43.388110,-0.675720,-0.688721,-0.118286,0.005920,-0.005310,-0.006226,0.469116,-0.593018,0.683716,0.014343,-0.000061,-0.001404
121847,1.0,christof,1.0,ankle,2.0,43.388460,-0.661804,-0.716248,-0.113647,0.005615,-0.006165,-0.005737,0.467163,-0.581665,0.703979,0.015015,-0.000366,-0.000610
121848,1.0,christof,1.0,ankle,2.0,43.388810,-0.665955,-0.705750,-0.110474,0.004395,-0.006958,-0.005615,0.468567,-0.573425,0.701965,0.015442,-0.000305,-0.000244


In [17]:
bioz_df.tt5k

0           0.089958
1           0.090503
2           0.091016
3           0.091561
4           0.092074
             ...    
296215    157.731291
296216    157.731803
296217    157.732349
296218    157.732861
296219    157.733407
Name: tt5k, Length: 296220, dtype: float64

In [22]:
fig, ax = plt.subplots()
ax.plot(bioz_df.tt5k, bioz_df.R100k)
ax.plot(bioz_df.tt100k, bioz_df.R100k)
plt.show()

In [27]:
(1/(imu_df.tt[2] - imu_df.tt[1]))/60

49.95121951219485

Find heel strikes and segment cycles

In [ ]:
def load_combined_dataset(subjects = [1,2,3,4,5], save = True):
    # IMPLEMENT 
    # for concatenating together saved data files 
    
    
    #loop through all subject file names
    for sub in subjects:
        # switch between doing no exo and doing exo files 
        
        
        # declare filename
        fileName = 'biomech_s' + subjNumber + '_' + collectionType
        
        # preprocess each bioz file
        preprocess_bioZ('Data/kinematicsData/exo_study_amro/' + fileName)
    
    # Whether to save data structure or not
    if save == True:
        
        
        


In [29]:
# Plot data to view - baseline
fig, ax = plt.subplots(3)
ax[0].plot(base_datasets[0].tissue_R[:,0])
ax[0].set_title('5k')
ax[1].plot(base_datasets[0].tt_ZZ, base_datasets[0].tissue_R[:,1])
ax[1].set_title('100k')
ax[2].plot(base_datasets[0].tt_ZZ, base_datasets[0].tissue_R[:,0] / base_datasets[0].tissue_R[:,1])
ax[2].set_title('ratio')
plt.show()

In [44]:
# baseline segmentation
baseline = []
baseline.append(base_datasets[0].tissue_R[8800:19550,:])
baseline.append(base_datasets[0].tissue_R[26400:36450,:])
baseline.append(base_datasets[0].tissue_R[45900:36450,:])
baseline.append(base_datasets[0].tissue_R[70000:130000,:])
np.shape(baseline[3])

(60000, 2)

(60000, 2)

Experimenting with ICA - take both frequencies and decompose into two components

In [46]:
# trying out ICA on dataset - lots of hyperparameters to play with
transformer = dc.FastICA(n_components=2)
baseline_transformed = transformer.fit_transform(baseline[3])
np.shape(baseline_transformed)

fig, ax = plt.subplots(2)
ax[0].plot(baseline_transformed[:,0])
ax[1].plot(baseline_transformed[:,1])
plt.show()

from sklearn.datasets import load_digits
from sklearn.decomposition import FastICA
X, _ = load_digits(return_X_y=True)
transformer = FastICA(n_components=7,random_state=0,whiten='unit-variance')
X_transformed = transformer.fit_transform(X)
X_transformed.shape